In [1]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

In [2]:
NFOLDS = 5

CACHE_PATH = '../input/dropoutrate01hiddensize128'

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.1)

        dropout_rate = 0.1
        hidden_size = 128
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [4]:
device = torch.device("cuda:0")
resnet_model_list = []
for _fold in range(NFOLDS):
    torch.cuda.empty_cache()
    model = Model()
    model.to(device)
    model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    resnet_model_list.append(model)
    
resnet_model_list=resnet_model_list[-2:]

In [5]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

In [6]:
keras_f_mean = np.load('../input/kerasfmeanonline/keras_f_mean_online.npy')

In [7]:
from keras.models import load_model
keras_model = load_model('../input/newkeras/model.h5')

In [8]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

test_df_columns = ['weight'] + [f'feature_{i}' for i in range(130)] + ['date']
index_features = [n for n, col in enumerate(test_df_columns) if col in feat_cols]

import gc
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].values[0]>0:
        
        resnet_x_tt = test_df.values[0][index_features].reshape(1, -1)
        if np.isnan(resnet_x_tt.sum()):
            resnet_x_tt = np.nan_to_num(resnet_x_tt) + np.isnan(resnet_x_tt) * f_mean
        cross_41_42_43 =resnet_x_tt[:, 41] +resnet_x_tt[:, 42] +resnet_x_tt[:, 43]
        cross_1_2 =resnet_x_tt[:, 1] / (resnet_x_tt[:, 2] + 1e-5)
        feature_inp = np.concatenate((
            resnet_x_tt,
            np.array(cross_41_42_43).reshape(resnet_x_tt.shape[0], 1),
            np.array(cross_1_2).reshape(resnet_x_tt.shape[0], 1),
            ), axis=1)
        resnet_pred = np.zeros((1, len(target_cols)))
        for model in resnet_model_list:
            resnet_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / 2
        resnet_pred = np.median(resnet_pred)
        
        keras_x_tt = test_df.values[0][index_features].reshape(1, -1)
        if np.isnan(keras_x_tt.sum()):
            keras_x_tt = np.nan_to_num(keras_x_tt) + np.isnan(keras_x_tt) * keras_f_mean
        keras_pred = keras_model(keras_x_tt, training = False).numpy() 
        keras_pred =  np.median(keras_pred)
        
        final_pred = 0.48*resnet_pred + 0.52*keras_pred 
        pred_df.action = int(final_pred >= 0.5) 
    else:
        pred_df['action'].values[0] = 0
    env.predict(pred_df)

15219it [04:53, 51.80it/s]
